In [26]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [22]:

def idw_interpolation(gdf, grid_gdf, radius, exponent, vname=''):
    """
    Realiza interpolación mediante el método IDW y agrega una columna con las predicciones al GeoDataFrame de la grilla.

    Args:
        gdf (geopandas.GeoDataFrame): DataFrame con los datos.
        grid_gdf (geopandas.GeoDataFrame): GeoDataFrame con la grilla de puntos a interpolar.
        radius (float): Radio de influencia.
        exponent (float): Exponente de la ponderación.
        vname (str): Nombre de la columna en gdf que contiene los valores a interpolar.

    Returns:
        geopandas.GeoDataFrame: GeoDataFrame con la geometría de la grilla y una nueva columna de predicciones.
    """
    # Crear un nuevo GeoDataFrame solo con la geometría de la grilla
    interpolated_gdf = gpd.GeoDataFrame({'geometry': grid_gdf['geometry'], 'obs': grid_gdf[vname]})
    
    # Iterar sobre los puntos de la grilla
    for idx, point in interpolated_gdf.iterrows():
        distances = gdf.distance(point.geometry)
        weights = 1 / (distances + 0.000001) ** exponent  # Se agrega un pequeño valor para evitar divisiones por cero
        interpolated_value = np.sum(weights * gdf[vname]) / np.sum(weights)
        interpolated_gdf.loc[idx, 'pred'] = interpolated_value
    
    return interpolated_gdf


def average_nearest_neighbor_distance(gdf):
    """
    Calcula la distancia promedio al vecino más cercano para un GeoDataFrame de puntos.

    Args:
        gdf (geopandas.GeoDataFrame): GeoDataFrame que contiene geometrías de puntos.

    Returns:
        float: Distancia promedio al vecino más cercano.
    """
    # Extraer las coordenadas de los puntos en el GeoDataFrame
    points = gdf.geometry.apply(lambda point: (point.x, point.y)).tolist()

    # Construir el árbol KD
    tree = KDTree(points)

    # Calcular la distancia al vecino más cercano para cada punto
    distances, _ = tree.query(points, k=2)  # k=2 para encontrar el vecino más cercano (el punto en sí mismo es el más cercano)

    # La distancia al vecino más cercano de cada punto es la segunda columna de la matriz distances
    nearest_distances = distances[:, 1]

    # Calcular la distancia promedio al vecino más cercano
    average_nearest_distance = nearest_distances.mean()

    return average_nearest_distance

In [29]:
# Cargar tus datos en un GeoDataFrame
data = gpd.read_file(filename="Data/ECa.gpkg",layer="EC_field_04")
folds = 5

In [30]:
X = data['geometry']
kf = KFold(n_splits=folds,shuffle=True, random_state=701408733)
kf.get_n_splits(X)
dist_nn = average_nearest_neighbor_distance(data)
idps = np.arange(1, 3.25, 0.25)
best_rmse, best_idp, best_prediction = np.inf, None, None
for idp in idps:
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        if i==0:
            temp_gdf = idw_interpolation(gdf=data.iloc[train_index],grid_gdf=data.iloc[test_index],vname='EC30',exponent=idp,radius=dist_nn*10)
        else:  
            trained_gdf = idw_interpolation(gdf=data.iloc[train_index],grid_gdf=data.iloc[test_index],vname='EC30',exponent=idp,radius=dist_nn*10)
            temp_gdf = pd.concat([temp_gdf,trained_gdf])
    # Calculating the RMSE using the observed and predicted values ​​in the DataFrame
    rmse = np.sqrt(mean_squared_error(temp_gdf['obs'], temp_gdf['pred']))
    print("idp:",round(idp,2)," - rmse:",round(rmse,2))
    if best_rmse>rmse:
        best_rmse, best_idp, best_prediction=rmse, idp, temp_gdf

idp: 1.0  - rmse: 4.065500333868567
idp: 1.25  - rmse: 3.758416223502052
idp: 1.5  - rmse: 3.3765227351924407
idp: 1.75  - rmse: 2.98349223269883
idp: 2.0  - rmse: 2.666965079886286
idp: 2.25  - rmse: 2.467366772827795
idp: 2.5  - rmse: 2.364355930380039
idp: 2.75  - rmse: 2.318935716481858
idp: 3.0  - rmse: 2.30224011360094


In [28]:
array = np.arange(1, 3.25, 0.25)
print(array)

[1.   1.25 1.5  1.75 2.   2.25 2.5  2.75 3.  ]


In [24]:
print(temp_gdf)

                            geometry   obs       pred
4     POINT (301919.415 5817068.537)  27.7  30.480018
5     POINT (301920.790 5817070.014)  26.5  26.829074
8     POINT (301925.427 5817075.122)  25.5  26.138285
16    POINT (301940.188 5817091.688)  19.4  19.917757
25    POINT (301966.960 5817121.867)  19.9  17.738698
...                              ...   ...        ...
3021  POINT (301840.127 5817898.604)  16.6  18.205726
3022  POINT (301842.954 5817902.003)  17.6  17.780893
3030  POINT (301869.268 5817933.059)  18.2  19.661280
3032  POINT (301879.196 5817941.736)  21.6  20.898663
3038  POINT (301904.973 5817943.800)  19.2  23.005567

[3039 rows x 3 columns]
